In [413]:
import numpy as np 
import math 

## Dados de entrada

In [414]:
d = ''
lim_esc = 800
tensao_ruptura = 1200
ciclos = 100000
p_max = 9000
montagem = 'fixação permanente'

aco = 'usinado_ou_laminado_frio'
fat_seg = 1
conf = 50
temp = 80
carga = 'axial'
kp_linha = 2.8

## Limite de resistência a fadiga (Cálculo dos K's)

### Acabamento superficial (Ka)

In [415]:
def ka(aco):
  if aco == 'forjado':
    a = 272
    b = -0.995
    ka = a * (tensao_ruptura ** b)
  elif aco == 'laminado_quente':
      a = 57.7
      b = -0.718
      ka = a * (tensao_ruptura ** b)
  elif aco == 'usinado_ou_laminado_frio':
      a = 4.51
      b = -0.265
      ka = a * ((tensao_ruptura/1e6) ** b)
  elif aco == 'retificado':
      a = 1.58
      b = -0.085
      ka = a * (tensao_ruptura ** b)
  elif aco == 'espelhado':
      a = 1
      b = 0
      ka = a * (tensao_ruptura ** b)
  return ka

In [416]:
if ka(aco) > 1:
  ka = 1
else:
  ka = ka(aco)

In [417]:
ka

1

### Tamanho da peça (Kb)

In [418]:
def kb(d):
  if carga == 'axial':
    if d == '':
      kb = 0.85
    else:
      kb = 1
  else:
    if d == '':
      kb = 0.85

    elif 2.79 <=  d <= 51:
        kb = (d / 7.62)**-0.107

    elif 51 < d <= 254:
        kb = 1.51 * (d**-0.157)
  
  return kb

In [419]:
if kb(d) > 1:
  kb = 1
else:
  kb = kb(d)

In [420]:
kb

0.85

### Confiabilidade (Kc)

In [421]:
def kc(conf):
  if conf == 50:
    kc = 1
    
  elif conf == 90:
      kc = 0.897

  elif conf == 95:
      kc = 0.868

  elif conf == 98:
      kc = 0.836

  elif conf == 99:
      kc = 0.814

  elif conf == 99.99:
      kc = 0.702

  elif conf == 99.9999:
      kc = 0.620

  elif conf == 99.99999:
      kc = 0.584

  return kc

In [422]:
if kc(conf) > 1:
  kc = 1
else:
  kc = kc(conf)

In [423]:
kc

1

### Temperatura (Kd)

In [424]:
def kd(temp):
  kd = 0.9877 + (0.6507 * (10**-3) * temp) - (0.3414 * (10**-5) * (temp**2)) + (0.5621 * (10**-8) * (temp**3)) - (6.246 * (10**-12) * (temp**4))

  return kd

In [425]:
if kd(temp) > 1:
  kd = 1
else:
  kd = kd(temp)

In [426]:
kd

1

### Tipo de carga (Ke)

In [427]:
def ke(carga):
  if carga == 'flexao':
    ke = 1
    
  elif carga == 'axial':
      ke = 0.85
      
  elif carga == 'torcao':
      ke = 0.577
      
  return ke

In [428]:
  ke(carga)

0.85

### Fatores diversos (Kf)

In [429]:
kf = 1

In [430]:
#determinação dos limites de resistência para o cálculo das tensões
lim_normal_res = ka* 1 * kc * kd * 0.85 * kf * 0.5 * tensao_ruptura

print(lim_normal_res)

510.0


## Resistência a fadiga para vida finita 

In [431]:
# def res_fadiga(tensao_ruptura,lim_normal_res):
#   x = (0.90 * tensao_ruptura) / lim_normal_res
#   m = -1/3 *(math.log(x,10))

#   y = ((0.9 * ((tensao_ruptura)**2)) / lim_normal_res)
#   b = math.log(y,10)

#   res_fadiga = round((10**b) * (ciclos**m),2)

#   return res_fadiga

In [432]:
# res_fadiga(tensao_ruptura,lim_normal_res)

In [433]:
# def kp(d):
  

In [434]:
#Cálculo da resistência de prova
rp = lim_esc * 0.855

#Cálculo da Força Inicial
if montagem == 'fixação permanente':
    def fi(a):
      return (0.9 * rp * a)
else:
    def fi(a):
      return (0.75 * rp * a)

In [435]:
def tensao_variavel(a):
  return (1/9 * ( (kp_linha* p_max)/ (2*a)))

In [436]:
def tensao_media(a):
  return tensao_variavel(a) + ((fi(a))/(a))

In [437]:
#Determinação da função utilizada no método de Newton
def f(a):
  return 1 - (((fat_seg * tensao_variavel(a))/(lim_normal_res))**2)  - (((fat_seg * tensao_media(a))/(lim_esc))**2) 

In [438]:
#Cálculo da derivada da função
def flin(f, a):  
  if a == 0:
    a = 1
    h = a / 5471
    flin = (1/(12*h)) * ( f(a- 2*h) - 8*f(a - h) + 8*f(a+h) - f(a + 2*h) )
  else:
    h = a / 5471
    flin = (1/(12*h)) * ( f(a- 2*h) - 8*f(a - h) + 8*f(a + h) - f(a + 2*h) )
  return flin


In [439]:
#Cálculo do valor inicial da área
a0 = p_max /lim_esc
print(a0)

11.25


In [440]:
#Método de newton
def newton(f, a0, epsilon, maxIter=20):
  if abs(f(a0)) <= epsilon:
    return (a0)

  print("k\t a0\t\t f(a0)") #cabeçalho da tabela que retornará com os valores embaixo. \t é o espaçamento que dou para ficar visual.
  
  k=1
  while k <= maxIter:
    a1 = a0 - f(a0)/flin(f, a0)
      
    print("%a\t%e\t%e" % (k, a1,f(a1)))

    if abs(f(a1)) <= epsilon:
      return a1      
    
    a0 = a1
    k = k + 1
  
  print('ERRO: Número máximo de iterações atingido')
  
  return a1  

In [441]:
#Resposta
raiz =  newton(f, a0, 10**-9, 20)

print(raiz)

k	 a0		 f(a0)
1	8.907184e+00	-2.807984e-02
2	9.346339e+00	-1.614299e-03
3	9.374763e+00	-6.009870e-06
4	9.374870e+00	-8.394074e-11
9.374869756682752
